In [1]:
# Install dependencies
!pip install transformers torch pandas keybert scikit-learn tqdm rouge_score bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1d64048e0ae4fd3f92e69b9fbabc35bfb6b3140b1f274a2e8d418978c29a8b6d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from keybert import KeyBERT
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
import os

# Preprocess dataset to extract title keywords
def preprocess_dataset(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    kw_model = KeyBERT()
    
    def extract_keywords(title):
        try:
            keywords = kw_model.extract_keywords(
                title, keyphrase_ngram_range=(1, 7), top_n=10
            )
            return ' '.join([kw[0] for kw in keywords])
        except:
            return ''
    
    df['title_keywords'] = df['title'].apply(extract_keywords)
    df.to_csv(output_csv, index=False)
    return df

# Custom Dataset for RoBERTa
class KeywordDataset(Dataset):
    def __init__(self, csv_file, tokenizer, mlb, max_length=512):
        self.df = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.mlb = mlb
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        abstract = str(self.df.iloc[idx]['abstract'])
        keywords = str(self.df.iloc[idx]['title_keywords']).split()

        encoding = self.tokenizer(
            abstract,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        labels = self.mlb.transform([keywords])[0]
        labels = torch.tensor(labels, dtype=torch.float32)

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': labels
        }

# Training Function
def train_roberta(model, train_loader, val_loader, device, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            total_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss:.4f}")

# Main Execution
if __name__ == "__main__":
    # File paths
    train_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/train.csv'
    val_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/val.csv'
    processed_train_csv = '/kaggle/working/processed_train.csv'
    processed_val_csv = '/kaggle/working/processed_val.csv'

    # Preprocess datasets
    preprocess_dataset(train_csv, processed_train_csv)
    preprocess_dataset(val_csv, processed_val_csv)

    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load keyword vocabulary
    train_data = pd.read_csv(processed_train_csv)
    train_keywords = train_data['title_keywords'].apply(lambda x: x.split() if x else []).values
    # Limit vocabulary size to avoid memory issues
    keyword_counts = {}
    for sublist in train_keywords:
        for kw in sublist:
            keyword_counts[kw] = keyword_counts.get(kw, 0) + 1
    top_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:1000]
    keyword_vocab = [kw for kw, _ in top_keywords]
    mlb = MultiLabelBinarizer(classes=keyword_vocab)
    # Fit the MultiLabelBinarizer with training keywords
    mlb.fit(train_keywords)

    # Initialize tokenizer and datasets
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    train_dataset = KeywordDataset(processed_train_csv, tokenizer, mlb)
    val_dataset = KeywordDataset(processed_val_csv, tokenizer, mlb)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    # Initialize RoBERTa model
    model = AutoModelForSequenceClassification.from_pretrained(
        'roberta-base',
        num_labels=len(keyword_vocab),
        problem_type='multi_label_classification'
    ).to(device)

    # Train RoBERTa
    train_roberta(model, train_loader, val_loader, device, epochs=3, lr=2e-5)

    # Save the model
    os.makedirs('/kaggle/working/finetuned_roberta_keywords', exist_ok=True)
    model.save_pretrained('/kaggle/working/finetuned_roberta_keywords')
    tokenizer.save_pretrained('/kaggle/working/finetuned_roberta_keywords')
    # Save keyword vocabulary
    with open('/kaggle/working/keyword_vocab.txt', 'w') as f:
        f.write('\n'.join(keyword_vocab))

2025-05-15 14:55:41.024331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747320941.208792      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747320941.258302      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Epoch 2, Average Validation Loss: 0.0435


Training Epoch 3: 100%|██████████| 223/223 [02:09<00:00,  1.73it/s]


Epoch 3, Average Training Loss: 0.0411


Validation Epoch 3: 100%|██████████| 28/28 [00:03<00:00,  7.02it/s]


Epoch 3, Average Validation Loss: 0.0359


In [3]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BartTokenizer, BartForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
import os
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Preprocess dataset to extract title keywords
def preprocess_dataset(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    kw_model = KeyBERT()
    
    def extract_keywords(title):
        try:
            keywords = kw_model.extract_keywords(
                title, keyphrase_ngram_range=(1, 7), top_n=10
            )
            return ' '.join([kw[0] for kw in keywords])
        except:
            return ''
    
    df['title_keywords'] = df['title'].apply(extract_keywords)
    df.to_csv(output_csv, index=False)
    return df

# RoBERTa-based keyword prediction function
def predict_keywords(abstract, roberta_model, roberta_tokenizer, keyword_vocab, max_len=512, device='cuda'):
    roberta_model.eval()
    encoding = roberta_tokenizer(
        abstract,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        outputs = roberta_model(
            input_ids=encoding['input_ids'].to(device),
            attention_mask=encoding['attention_mask'].to(device)
        )
    
    logits = outputs.logits
    predictions = torch.sigmoid(logits).squeeze()
    predicted_keywords = [keyword_vocab[i] for i, prob in enumerate(predictions) if prob > 0.5]
    return predicted_keywords

# Custom Dataset
class ScientificPaperDataset(Dataset):
    def __init__(self, csv_file, bart_tokenizer, roberta_model, roberta_tokenizer, keyword_vocab, max_length=512, device='cuda'):
        self.df = pd.read_csv(csv_file)
        self.bart_tokenizer = bart_tokenizer
        self.roberta_model = roberta_model
        self.roberta_tokenizer = roberta_tokenizer
        self.keyword_vocab = keyword_vocab
        self.max_length = max_length
        self.device = device

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        abstract = str(self.df.iloc[idx]['abstract'])
        title = str(self.df.iloc[idx]['title'])
        title_keywords = str(self.df.iloc[idx]['title_keywords'])

        # Predict abstract keywords using RoBERTa
        abstract_keywords = predict_keywords(abstract, self.roberta_model, self.roberta_tokenizer, self.keyword_vocab, device=self.device)
        abstract_keywords_text = ' '.join(abstract_keywords) if abstract_keywords else ''

        # Input for BART: RoBERTa-predicted keywords + abstract
        input_text = f"Keywords: {abstract_keywords_text} [SEP] Abstract: {abstract}"

        input_encoding = self.bart_tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        title_encoding = self.bart_tokenizer(
            title,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'title_input_ids': title_encoding['input_ids'].squeeze(),
            'title_attention_mask': title_encoding['attention_mask'].squeeze(),
            'title_keywords': title_keywords,
            'abstract_keywords': abstract_keywords_text,
            'title': title
        }

# Multi-task BART Model
class MultiTaskBART(nn.Module):
    def __init__(self, bart_model_name='facebook/bart-base'):
        super(MultiTaskBART, self).__init__()
        self.bart = BartForConditionalGeneration.from_pretrained(bart_model_name)
        self.tokenizer = BartTokenizer.from_pretrained(bart_model_name)
        self.tfidf = TfidfVectorizer(max_features=3000)

    def forward(self, input_ids, attention_mask, title_input_ids, title_keywords, abstract_keywords):
        outputs = self.bart(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=title_input_ids
        )
        title_loss = outputs.loss

        try:
            keyword_vectors = self.tfidf.fit_transform([title_keywords, abstract_keywords]).toarray()
            title_kw_vector = torch.tensor(keyword_vectors[0], dtype=torch.float32)
            abstract_kw_vector = torch.tensor(keyword_vectors[1], dtype=torch.float32)
            cosine_sim = nn.functional.cosine_similarity(title_kw_vector, abstract_kw_vector, dim=0)
            keyword_loss = 1 - cosine_sim
        except:
            keyword_loss = torch.tensor(0.0, device=input_ids.device)

        return title_loss, keyword_loss

# Training and Validation Function
def train_and_validate(model, train_loader, val_loader, device, epochs=3, lr=5e-5, title_loss_weight=1.0, keyword_loss_weight=0.5):
    optimizer = AdamW(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            title_input_ids = batch['title_input_ids'].to(device)
            title_keywords = batch['title_keywords']
            abstract_keywords = batch['abstract_keywords']

            title_loss, keyword_loss = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                title_input_ids=title_input_ids,
                title_keywords=title_keywords,
                abstract_keywords=abstract_keywords
            )

            loss = title_loss_weight * title_loss + keyword_loss_weight * keyword_loss
            total_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                title_input_ids = batch['title_input_ids'].to(device)
                title_keywords = batch['title_keywords']
                abstract_keywords = batch['abstract_keywords']

                title_loss, keyword_loss = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    title_input_ids=title_input_ids,
                    title_keywords=title_keywords,
                    abstract_keywords=abstract_keywords
                )

                loss = title_loss_weight * title_loss + keyword_loss_weight * keyword_loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss:.4f}")

# Evaluation Function with ROUGE and BERTScore
def evaluate_model(model, test_loader, device):
    model.eval()
    generated_titles = []
    reference_titles = []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'rougeLsum': []}

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            titles = batch['title']

            # Generate titles
            generated_ids = model.bart.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=32,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True
            )
            generated_title = [model.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

            generated_titles.extend(generated_title)
            reference_titles.extend(titles)

            # Compute ROUGE scores
            for gen, ref in zip(generated_title, titles):
                scores = scorer.score(ref, gen)
                for metric in rouge_scores:
                    rouge_scores[metric].append(scores[metric].fmeasure)

    # Compute BERTScore
    try:
        P, R, F1 = bert_score(generated_titles, reference_titles, lang="en", verbose=True)
        bertscore_f1 = F1.mean().item()
    except:
        bertscore_f1 = 0.0

    # Average ROUGE scores
    avg_rouge = {metric: np.mean(scores) for metric, scores in rouge_scores.items()}

    # Print results
    print("\nEvaluation Results:")
    print(f"ROUGE-1: {avg_rouge['rouge1']:.4f}")
    print(f"ROUGE-2: {avg_rouge['rouge2']:.4f}")
    print(f"ROUGE-L: {avg_rouge['rougeL']:.4f}")
    print(f"ROUGE-Lsum: {avg_rouge['rougeLsum']:.4f}")
    print(f"BERTScore F1: {bertscore_f1:.4f}")

    # Save generated titles
    results_df = pd.DataFrame({
        'reference_title': reference_titles,
        'generated_title': generated_titles
    })
    results_df.to_csv('/kaggle/working/generated_titles.csv', index=False)

    return avg_rouge, bertscore_f1

# Main Execution
if __name__ == "__main__":
    # File paths
    train_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/train.csv'
    val_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/val.csv'
    test_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/test.csv'
    processed_train_csv = '/kaggle/working/processed_train.csv'
    processed_val_csv = '/kaggle/working/processed_val.csv'
    processed_test_csv = '/kaggle/working/processed_test.csv'

    # Preprocess datasets
    preprocess_dataset(train_csv, processed_train_csv)
    preprocess_dataset(val_csv, processed_val_csv)
    preprocess_dataset(test_csv, processed_test_csv)

    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load RoBERTa model and tokenizer for keyword prediction
    roberta_model_path = '/kaggle/working/finetuned_roberta_keywords'
    if not os.path.exists(roberta_model_path):
        raise FileNotFoundError(f"RoBERTa model directory {roberta_model_path} does not exist. Please run the fine-tuning script first.")
    roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_path).to(device)
    roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_path)

    # Load keyword vocabulary
    keyword_vocab_path = '/kaggle/working/keyword_vocab.txt'
    if not os.path.exists(keyword_vocab_path):
        raise FileNotFoundError(f"Keyword vocabulary file {keyword_vocab_path} does not exist. Please run the fine-tuning script first.")
    with open(keyword_vocab_path, 'r') as f:
        keyword_vocab = [line.strip() for line in f]

    # Initialize BART tokenizer and datasets
    bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
    train_dataset = ScientificPaperDataset(processed_train_csv, bart_tokenizer, roberta_model, roberta_tokenizer, keyword_vocab, device=device)
    val_dataset = ScientificPaperDataset(processed_val_csv, bart_tokenizer, roberta_model, roberta_tokenizer, keyword_vocab, device=device)
    test_dataset = ScientificPaperDataset(processed_test_csv, bart_tokenizer, roberta_model, roberta_tokenizer, keyword_vocab, device=device)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    # Initialize BART model
    bart_model = MultiTaskBART().to(device)

    # Train and validate
    train_and_validate(bart_model, train_loader, val_loader, device, epochs=3, lr=5e-5, 
                       title_loss_weight=1.0, keyword_loss_weight=0.5)

    # Evaluate on test set
    rouge_results, bertscore_result = evaluate_model(bart_model, test_loader, device)

    # Save the model
    bart_model.bart.save_pretrained('/kaggle/working/multi_task_bart_model')
    bart_tokenizer.save_pretrained('/kaggle/working/multi_task_bart_model')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.10 seconds, 202.20 sentences/sec

Evaluation Results:
ROUGE-1: 0.6498
ROUGE-2: 0.4490
ROUGE-L: 0.5835
ROUGE-Lsum: 0.5835
BERTScore F1: 0.9236


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
